In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install tensorflow_text 
!pip install transformers emoji
# !pip install ktrain

In [ ]:
from transformers import AutoTokenizer


In [ ]:
import pandas as pd
dataset = pd.read_excel("/content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model_data.xlsx")
dataset = dataset.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True) 
dataset = dataset[['p_message','category']]
print(len(dataset))
dataset.head()

29354


,p_message,category
0,المحاضرات هي حتى للتفكير وتنوير ... شكرا جزيلا!,education
1,هل يمكنك إصلاح شبكة هراءOmantel,telecom
2,قتل الساقين اليوم، حصلت على هذا الدجاج البرتقا...,food
3,لا أعرف أي شيء عن الدراجات.,transport
4,عدلوا شبكتكم لو سمحتوا \n @Omantel,telecom


In [ ]:
dataset.groupby('category').size()

category
business         2861
education        2212
entertainment    2232
fashion          2000
food             2238
health           2151
politics         2731
sports           2227
technology       2403
telecom          2323
tourism          1648
transport        2052
weather          2167
dtype: int64

In [ ]:
def get_label(label):
  if label == "business":
    return 0
  elif label == "education":
    return 1
  elif label == 'entertainment':
    return 2
  elif label == 'fashion':
    return 3
  elif label == 'food':
    return 4
  elif label == 'health':
    return 5
  elif label == 'politics':
    return 6
  elif label == 'sports':
    return 7
  elif label == 'technology':
    return 8
  elif label == 'telecom':
    return 9
  elif label == 'tourism':
    return 10
  elif label == 'transport':
    return 11
  elif label == 'weather':
    return 12

dataset['category'] = dataset.category.apply(to_int_sentiment)
dataset = dataset.dropna()
dataset['category'] = dataset.category.apply(int)

In [ ]:
dataset.isnull().sum()

p_message    0
category     0
dtype: int64

In [ ]:
from preprocessing import ArCleanText
ar_text_clean = ArCleanText()



In [ ]:
dataset['p_message'] = dataset['p_message'].apply(ar_text_clean.arabic_preprocessing)
dataset.head()


,p_message,category
0,المحاضرات هي حتي للتفكير وتنوير شكرا جزيلا,1
1,هل يمكنك اصلاح شبكه هراء,9
2,قتل الساقين اليوم حصلت علي هذا الدجاج البرتقال...,4
3,لا اعرف اي شيء عن الدراجات,11
4,عدلوا شبكتكم لو سمحتوا,9


In [ ]:
dataset.groupby('category').size()

category
0     2861
1     2212
2     2232
3     2000
4     2238
5     2151
6     2731
7     2227
8     2403
9     2323
10    1648
11    2052
12    2167
dtype: int64

In [ ]:
dataset.isnull().sum()

p_message    0
category     0
dtype: int64

In [ ]:
dataset.head()

,p_message,category
0,المحاضرات هي حتي للتفكير وتنوير شكرا جزيلا,1
1,هل يمكنك اصلاح شبكه هراء,9
2,قتل الساقين اليوم حصلت علي هذا الدجاج البرتقال...,4
3,لا اعرف اي شيء عن الدراجات,11
4,عدلوا شبكتكم لو سمحتوا,9


In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback


# Read data
# dataset = pd.read_excel("/content/drive/MyDrive/ar_general_sentiment_data.xlsx")
# dataset = dataset[['text','Sentiment']]

# Define pretrained tokenizer and model
model_name = "prajjwal1/bert-small"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=13)

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(dataset["p_message"])
y = list(dataset["Category"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)





Downloading:   0%|          | 0.00/334k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/509 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/46.6M [00:00<?, ?B/s]

Some weights of the model checkpoint at asafaya/bert-mini-arabic were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at asafaya/bert-mini-ar

In [ ]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='micro')
    precision = precision_score(y_true=labels, y_pred=pred,average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model
trainer.train()


***** Running training *****
  Num examples = 23396
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14625


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,1.675900,0.990627,0.794495,0.794495,0.794495,0.794495
1000,0.873900,0.681922,0.820995,0.820995,0.820995,0.820995
1500,0.680800,0.579760,0.840998,0.840998,0.840998,0.840998
2000,0.582800,0.567876,0.844589,0.844589,0.844589,0.844589
2500,0.532700,0.521185,0.855360,0.855360,0.855360,0.855360
3000,0.507800,0.514882,0.858950,0.858950,0.858950,0.858950
3500,0.385200,0.516726,0.869208,0.869208,0.869208,0.869208
4000,0.342600,0.526196,0.871431,0.871431,0.871431,0.871431
4500,0.389600,0.525640,0.873654,0.873654,0.873654,0.873654
5000,0.365600,0.529401,0.875876,0.875876,0.875876,0.875876


***** Running Evaluation *****
  Num examples = 5849
  Batch size = 8
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5849
  Batch size = 8
Saving model checkpoint to output/checkpoint-1000
Configuration saved in output/checkpoint-1000/config.json
Model weights saved in output/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5849
  Batch size = 8
Saving model checkpoint to output/checkpoint-1500
Configuration saved in output/checkpoint-1500/config.json
Model weights saved in output/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5849
  Batch size = 8
Saving model checkpoint to output/checkpoint-2000
Configuration saved in output/checkpoint-2000/config.json
Model weights saved in output/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Nu

TrainOutput(global_step=14625, training_loss=0.34502269438393096, metrics={'train_runtime': 2809.7635, 'train_samples_per_second': 41.633, 'train_steps_per_second': 5.205, 'total_flos': 1160268507033600.0, 'train_loss': 0.34502269438393096, 'epoch': 5.0})

In [ ]:
model_path = "/content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model"
trainer.save_model(model_path)

Saving model checkpoint to /content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model
Configuration saved in /content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model/config.json
Model weights saved in /content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model/pytorch_model.bin


In [ ]:
# Tokenize test data
x = ["إنه جيد جدًا في رياضة الكريكيت خاصة"]
X_test_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512) 
# Create torch dataset
test_dataset = Dataset(X_test_tokenized) 
# Load trained model
# model_path = "sentiment_model"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=13) 
# Define test trainer
test_trainer = Trainer(model) 
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset) 
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file /content/drive/MyDrive/Arabic Category Transformer Model/ar_category_model/config.json
Model config BertConfig {
  "_name_or_path": "asafaya/bert-mini-arabic",
  "_num_labels": 2,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "

In [ ]:
y_pred

array([ 7, 11, 10, ...,  3,  2,  5])

In [ ]:
y_pred

array([ 7, 11, 10, ...,  3,  2,  5])

In [ ]:
len(y_val)

5849

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
class_names = ["0","1","2","3","4","5","6","7","8","9","10","11","12"]
print(classification_report(y_val, y_pred, target_names=class_names))

              precision    recall  f1-score   support

           0       0.83      0.78      0.81       581
           1       0.90      0.82      0.86       444
           2       0.87      0.83      0.85       441
           3       0.96      0.96      0.96       383
           4       0.89      0.95      0.92       419
           5       0.80      0.82      0.81       444
           6       0.86      0.81      0.84       506
           7       0.90      0.95      0.92       460
           8       0.64      0.86      0.73       481
           9       0.90      0.76      0.82       485
          10       0.97      0.86      0.91       340
          11       0.90      0.90      0.90       418
          12       0.92      0.90      0.91       447

    accuracy                           0.86      5849
   macro avg       0.87      0.86      0.87      5849
weighted avg       0.87      0.86      0.86      5849



In [ ]:
 print(confusion_matrix(y_val, y_pred))

[[456   5   3   1   7   9  31   3  36  19   4   5   2]
 [ 14 364   9   2   3   8   5   0  31   1   0   1   6]
 [  1   4 368   2   6  13   4  12  21   4   1   3   2]
 [  0   1   1 367   2   2   0   3   4   0   0   3   0]
 [  1   1   1   0 400   8   1   2   1   1   0   1   2]
 [  6   8   5   2   8 365   4   9  24   1   0   5   7]
 [ 28   4   5   0   1  16 412   8  18   2   0   6   6]
 [  1   1   1   0   3   7   3 437   2   2   0   1   2]
 [ 15  11   9   2   5   8   2   3 415   5   0   5   1]
 [ 14   3  12   0   2  11   7   2  58 367   2   4   3]
 [ 10   1   5   1   3   1   6   1   6   6 292   6   2]
 [  4   0   2   0   3   4   1   1  21   1   1 378   2]
 [  0   2   4   5   5   4   2   5  13   0   1   3 403]]


In [ ]:
df = pd.DataFrame(X_val,columns =['p_message'])
df.head()

,p_message
0,الهناءي خبروا الشركتين الراءدتين محققات الملاي...
1,عذراء استراليا النقل بسيارات الاجره علي تل ال...
2,الهدف السياحي الذي حققته اسطنبول بفضل السياح ا...
3,وقد سجلت قيمه المضبوطات من البضاءع المهربه بزي...
4,ويقال ميدلسبره مقاضاه الدوري الانكليزي لكره ال...


In [ ]:
df['prediction'] = y_pred

In [ ]:
df.head()

,p_message,prediction
0,الهناءي خبروا الشركتين الراءدتين محققات الملاي...,7
1,عذراء استراليا النقل بسيارات الاجره علي تل ال...,11
2,الهدف السياحي الذي حققته اسطنبول بفضل السياح ا...,10
3,وقد سجلت قيمه المضبوطات من البضاءع المهربه بزي...,0
4,ويقال ميدلسبره مقاضاه الدوري الانكليزي لكره ال...,7


In [ ]:
df.groupby('prediction').size()

prediction
0     550
1     405
2     425
3     382
4     448
5     456
6     478
7     486
8     650
9     409
10    301
11    421
12    438
dtype: int64

In [ ]:
df.to_excel("/content/drive/MyDrive/prediction_data.xlsx")